In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models

from backdoor.poisons import BadNetsPoison

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = '/data'

In [3]:
normalizer = transforms.Compose([
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

denormalizer = transforms.Compose([
    transforms.Normalize(mean=[0., 0., 0.], std=[1/0.2023, 1/0.1994, 1/0.2010]),
    transforms.Normalize(mean=[-0.4914, -0.4822, -0.4465], std=[1., 1., 1.]),
])

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalizer
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    normalizer
])

In [4]:
trainset = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform_train)
testset = datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform_test)
poisoned_testset = BadNetsPoison(
    testset,
    target_class=5,
    poison_ratio=1,
    poison_type='dirty',
    trigger_img='trigger_10',
    trigger_size=3
)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)
poisoned_testloader = torch.utils.data.DataLoader(poisoned_testset, batch_size=100, shuffle=False, num_workers=2)

In [6]:
poisoned_net = models.resnet18(weights=None, num_classes=10).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(poisoned_net.parameters(), lr=0.001)

In [7]:
poisoned_net.load_state_dict(torch.load('cifar10_badnet_50.pth'))

<All keys matched successfully>

In [8]:
def test_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            preds = model(inputs).argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total

def evaluate_attack(model, clean_loader, poisoned_loader):

    clean_acc = test_model(model, clean_loader)
    attack_success_count = 0

    for inputs, labels, targets in poisoned_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        targets = targets.to(device)
        preds = model(inputs).argmax(dim=1)
        correct = (preds == targets).sum().item()
        attack_success_count += correct

    attack_success_rate = attack_success_count / len(poisoned_loader.dataset)
    return clean_acc, attack_success_rate


acc, asr = evaluate_attack(poisoned_net, testloader, poisoned_testloader)
print(f'Clean accuracy: {acc:.4f}, \nAttack success rate: {asr:.4f}')

Clean accuracy: 0.8329, 
Attack success rate: 0.9937


# STRIP Defense

In [9]:
from backdoor.defenses import STRIP

strip_defense = STRIP(poisoned_net, testset, transform_test)

In [10]:
strip_defense.detect()

100%|█████████████████████████████████████| 10000/10000 [29:03<00:00,  5.74it/s]


ValueError: multiclass format is not supported